## Set up

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

load_dotenv()

doublecheck_env("../.env")

OLLAMA_MODEL=****a3.2
OPENAI_API_KEY=****upAA
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=****.com
LANGSMITH_API_KEY=****7b12
LANGSMITH_PROJECT=****ject


## Models and Messages

Models docs: https://docs.langchain.com/oss/python/integrations/chat

Messages types:
- SystemMessage
- HumanMessage
- AIMessage
- ToolMessage

In [2]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

In [3]:
@tool
def get_stock_price(symbol: str) -> str:
    """Get today's stock price for a given symbol."""
    # dummy logic and price
    prices = {
        "BBCA": 9500,
        "BMRI": 7200,
        "TLKM": 3100,
    }
    price = prices.get(symbol.upper(), None)

    if price is None:
        return f"Symbol {symbol} not found."

    return f"The current price of {symbol} is {price}."

In [4]:
SYSTEM_PROMPT = "You are a holistic stock market analyst in Indonesia, you will provide data the user ask, you can use tools to get the data"


agent = create_agent(
    model="gpt-5-mini",
    tools=[get_stock_price],
    system_prompt=SYSTEM_PROMPT
)

In [5]:
human_msg = HumanMessage(content="Give price for BBCA and BMRI.")
result = agent.invoke({"messages": [human_msg]})

In [6]:
print(result["messages"][-1].content)

- BBCA: IDR 9,500 per share
- BMRI: IDR 7,200 per share

Prices are current as of 2025-12-07. Would you like intraday change, historical chart, or fundamentals for either stock?


In [7]:
result["messages"]

[HumanMessage(content='Give price for BBCA and BMRI.', additional_kwargs={}, response_metadata={}, id='d6117420-7518-4ff4-afcf-93924aa5c0e4'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 313, 'prompt_tokens': 167, 'total_tokens': 480, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CjzoLmqmlbFTFVqzdNCJkc9F1F3zO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019af6de-eba6-7ec3-960f-2c45cc6c8da3-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'BBCA'}, 'id': 'call_BGIVRamu95FDKgNsq6OXHptc', 'type': 'tool_call'}, {'name': 'get_stock_price', 'args': {'symbol': 'BMRI'}, 'id': 'call_OInpSmMXb2EzkXZ

In [8]:
for i in range(len(result["messages"])):
    print(type(result["messages"][i]))


<class 'langchain_core.messages.human.HumanMessage'>
<class 'langchain_core.messages.ai.AIMessage'>
<class 'langchain_core.messages.tool.ToolMessage'>
<class 'langchain_core.messages.tool.ToolMessage'>
<class 'langchain_core.messages.ai.AIMessage'>


In [9]:
for msg in result["messages"]:
    print(f"{msg.type.capitalize()}: {msg.content}\n")

Human: Give price for BBCA and BMRI.

Ai: 

Tool: The current price of BBCA is 9500.

Tool: The current price of BMRI is 7200.

Ai: - BBCA: IDR 9,500 per share
- BMRI: IDR 7,200 per share

Prices are current as of 2025-12-07. Would you like intraday change, historical chart, or fundamentals for either stock?



In [10]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Give price for BBCA and BMRI.
================================== Ai Message ==================================
Tool Calls:
  get_stock_price (call_BGIVRamu95FDKgNsq6OXHptc)
 Call ID: call_BGIVRamu95FDKgNsq6OXHptc
  Args:
    symbol: BBCA
  get_stock_price (call_OInpSmMXb2EzkXZPxiM8ZDr3)
 Call ID: call_OInpSmMXb2EzkXZPxiM8ZDr3
  Args:
    symbol: BMRI
================================= Tool Message =================================
Name: get_stock_price

The current price of BBCA is 9500.
================================= Tool Message =================================
Name: get_stock_price

The current price of BMRI is 7200.
================================== Ai Message ==================================

- BBCA: IDR 9,500 per share
- BMRI: IDR 7,200 per share

Prices are current as of 2025-12-07. Would you like intraday change, historical chart, or fundamentals for either stock?


## Streaming

In [11]:
for step in agent.stream(
    {"messages": [human_msg]},
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Give price for BBCA and BMRI.
================================== Ai Message ==================================
Tool Calls:
  get_stock_price (call_xfLizBRSf4XwWhypXhINtyRX)
 Call ID: call_xfLizBRSf4XwWhypXhINtyRX
  Args:
    symbol: BBCA
  get_stock_price (call_mAn0oGwM8W1lEchLGXpVsQk9)
 Call ID: call_mAn0oGwM8W1lEchLGXpVsQk9
  Args:
    symbol: BMRI
================================= Tool Message =================================
Name: get_stock_price

The current price of BMRI is 7200.
================================== Ai Message ==================================

Current prices (IDX):
- BBCA: IDR 9,500 per share
- BMRI: IDR 7,200 per share

Prices are the current quotes. Want intraday chart, recent change %, or historical data?


In [12]:
for token, metadata in agent.stream(
    {"messages": [human_msg]},
    stream_mode="messages"
):
    print(f"{token.content}", end="")

Symbol BBCA.JK not found.Symbol BMRI.JK not found.The current price of BBCA is 9500.The current price of BMRI is 7200.Here are the current prices I fetched:

- BBCA: 9,500 IDR per share  
- BMRI: 7,200 IDR per share

(From my price feed — let me know if you want timestamp, intraday chart, recent change %, or fundamentals for either stock.)

In [13]:
for step in agent.stream(
    {"messages": [human_msg]},
    stream_mode=["custom"] # ["custom", "values", "messages"]
):
    print(step)

## Tools & MCP

Tools provide Action part of ReAct

The reasoning node use the description to decide when to call the tools

In [14]:
from typing import Literal
from langchain_core.tools import tool

@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def number_calculator(
    a: float, b: float, operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.
            - `"multiply"`: Returns the product of `a` and `b`.
            - `"divide"`: Returns the result of `a / b`. Raises an error if `b` is zero.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🔎 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [15]:
agent = create_agent(
    model="gpt-5-mini",
    tools=[number_calculator],
    system_prompt="You are a helpful math assistant. Use the calculator tool to perform arithmetic operations when needed."
)

In [16]:
res = agent.invoke({"messages": [HumanMessage(content="What is 25 multiplied by 4, then add 10?")]})
print(res["messages"][-1].content)

🔎 Invoking calculator tool
🔎 Invoking calculator tool
25 × 4 = 100, then 100 + 10 = 110. So the answer is 110.


## Memory

- Short term memory vs Long term memory
- Cognitive layer, State layer etc

In [49]:
from langchain_community.utilities import SQLDatabase
from langchain_core.messages import HumanMessage

db = SQLDatabase.from_uri("sqlite:///studio/Chinook.db")

In [44]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [45]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQL query against the database and return the results."""

    try:
        runtime = get_runtime()
        db = runtime.context.db
        
        print("Runtime inside tool:", runtime)
        print("Runtime context:", runtime.context)

        result = db.run(query)
        return str(result)
    except Exception as e:
        return f"Error executing query: {e}"

In [46]:
SYSTEM_PROMPT = """You are a careful Data Analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows of output unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [47]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver()
)

In [50]:
question = "This is Frank Ralston, what was my total invoice amount?"
steps = []

for step in agent.stream(
    {"messages": [HumanMessage(content=question)]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Ralston, what was my total invoice amount?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_UcGWiAuY03lJr1s6KFJAnTUG)
 Call ID: call_UcGWiAuY03lJr1s6KFJAnTUG
  Args:
    query: SELECT c.CustomerId, c.FirstName, c.LastName, SUM(i.Total) AS total_amount
FROM customers c
JOIN invoices i ON c.CustomerId = i.CustomerId
WHERE c.FirstName = 'Frank' AND c.LastName = 'Ralston'
GROUP BY c.CustomerId, c.FirstName, c.LastName
LIMIT 5;
Runtime inside tool: Runtime(context=RuntimeContext(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018B38F938C0>), store=None, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x0000018B16619EE0>, previous=None)
Runtime context: RuntimeContext(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000018B38F938C0>)
================================= To

In [27]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [HumanMessage(content=question)]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values"
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_y3pbD9ny0ebcSnkLyCr7uBCp)
 Call ID: call_y3pbD9ny0ebcSnkLyCr7uBCp
  Args:
    query: SELECT t.TrackId, t.Name AS Title, i.InvoiceId, i.InvoiceDate, il.UnitPrice, il.Quantity
FROM Customer c
JOIN Invoice i ON c.CustomerId = i.CustomerId
JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
JOIN Track t ON il.TrackId = t.TrackId
WHERE c.FirstName = 'Frank' AND c.LastName = 'Ralston'
ORDER BY i.InvoiceDate, i.InvoiceId, il.InvoiceLineId
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[(3018, 'Sunday Bloody Sunday', 92, '2010-02-08 00:00:00', 0.99, 1), (3020, "New Year's Day", 92, '2010-02-08 00:00:00', 0.99, 1), (3347, 'Meet Kevin Johnson', 103, '2010-03-21 00:00:00', 1.99, 1), (3356, 'Muita Bobeira', 103, '2010-03-21 0

## Structured output

In [28]:
from langchain.agents import create_agent
from typing_extensions import TypedDict
from pydantic import BaseModel

In [30]:
class ContactInfo(TypedDict):
    name: str
    email: str
    phone: str


agent = create_agent(model="gpt-5-mini", response_format=ContactInfo)

recorded_conversation = """We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]

{'name': 'John Doe', 'email': 'john@example.com', 'phone': '555-123-4567'}

In [31]:
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str


agent = create_agent(model="gpt-5-mini", response_format=ContactInfo)

recorded_conversation = """We talked with John Doe. He works over at Example. His number is, let's see, 
five, five, five, one two three, four, five, six seven. Did you get that?
And, his email was john at example.com. He wanted to order 50 boxes of Captain Crunch."""

result = agent.invoke(
    {"messages": [{"role": "user", "content": recorded_conversation}]}
)

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='555-123-4567')

## Middleware: Dynamic Prompt

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langgraph.runtime import get_runtime
from dataclasses import dataclass

db = SQLDatabase.from_uri("sqlite:///studio/Chinook.db")

@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQL query against the database and return the results."""

    try:
        runtime = get_runtime(RuntimeContext)
        db = runtime.context.db

        result = db.run(query)
        return str(result)
    except Exception as e:
        return f"Error executing query: {e}"
    
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [34]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt

@dynamic_prompt
def system_prompt_middleware(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Here are non employee so limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track. And you are not allowed to execute those tables whatever the user asks"
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

In [35]:
from langchain.agents import create_agent

agent = create_agent(
    model = "gpt-5-mini",
    tools = [execute_sql],
    middleware=[system_prompt_middleware],
    context_schema=RuntimeContext,
)

In [36]:
question = "What is the most costly purchase by Frank Ralston?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Ralston?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_q7VvLpDpP2C6BtoGIRyEWh3d)
 Call ID: call_q7VvLpDpP2C6BtoGIRyEWh3d
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.BillingAddress, i.BillingCity, i.BillingState, i.BillingCountry, i.Total
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
WHERE c.FirstName = 'Frank' AND c.LastName = 'Ralston'
ORDER BY i.Total DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(103, '2010-03-21 00:00:00', '162 E Superior Street', 'Chicago', 'IL', 'USA', 15.86)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_NQ3ajlgqQGTlqainV0odc3iQ)
 Call ID: call_NQ3ajlgqQGTlqainV0odc3iQ
  Args:
    query: SELECT InvoiceLineId, Invo

In [33]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_CxxbdSy47vmeI4XpcSEUoDBo)
 Call ID: call_CxxbdSy47vmeI4XpcSEUoDBo
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_yvodAk6Ip3yepdTlICzS9hqc)
 Call ID: call_yvodAk6Ip3yepdTlICzS9hqc
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.BillingCountry, SUM(il.UnitPrice * il.Quantity) AS total
FROM Customer c
JOIN Invoice i ON c.CustomerId = i

## Middleware Human in the Loop

In [4]:

from langchain_community.utilities import SQLDatabase
from langchain_core.tools import tool
from langgraph.runtime import get_runtime
from dataclasses import dataclass

db = SQLDatabase.from_uri("sqlite:///studio/Chinook.db")

@dataclass
class RuntimeContext:
    db: SQLDatabase
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQL query against the database and return the results."""

    try:
        runtime = get_runtime(RuntimeContext)
        db = runtime.context.db

        result = db.run(query)
        return str(result)
    except Exception as e:
        return f"Error executing query: {e}"
    
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If the database is offline, ask user to try again later without further comment.
"""

In [5]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-5-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT_TEMPLATE,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"execute_sql": {"allowed_decisions": ["approve", "reject"]}}
        )
    ]
)

In [6]:
from langgraph.types import Command

question = "What are the names of all the employees?"
config = {"configurable": {"thread_id": "1"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )

    result = agent.invoke(
        Command(
            resume={
                "decisions": [{"type": "reject", "message": "the database is offline."}]
            }
        ),
        config=config,  # Same thread ID to resume the paused conversation
        context=RuntimeContext(db=db),
    )
    print(f"\033[1;3;31m{80 * '-'}\033[0m")

print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%' AND (lower(name) LIKE '%emp%' OR lower(name) LIKE '%employee%' OR lower(name) LIKE '%staff%' OR lower(name) LIKE '%person%' OR lower(name) LIKE '%people%') LIMIT 5;"}
--------------------------------------------------------------------------------
The database is currently offline. Please try again later.


In [7]:
config = {"configurable": {"thread_id": "2"}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    context=RuntimeContext(db=db)
)

while "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print(f"\033[1;3;31m{80 * '-'}\033[0m")
    print(
        f"\033[1;3;31m Interrupt:{description}\033[0m"
    )
    
    result = agent.invoke(
        Command(
            resume={"decisions": [{"type": "approve"}]}
        ),
        config=config,  # Same thread ID to resume the paused conversation
        context=RuntimeContext(db=db),
    )

for msg in result["messages"]:
    msg.pretty_print()

--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT COALESCE(first_name || ' ' || last_name, name, full_name) AS employee_name FROM employees LIMIT 5;"}
--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT name, type FROM sqlite_master WHERE type IN ('table','view') LIMIT 5;"}
--------------------------------------------------------------------------------
 Interrupt:Tool execution requires approval

Tool: execute_sql
Args: {'query': "SELECT FirstName || ' ' || LastName AS employee_name FROM Employee LIMIT 5;"}
================================ Human Message =================================

What are the names of all the employees?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_rUhCzRMZRSAFBrFC